In [1]:
import graphlab

A newer version of GraphLab Create (v1.10.1) is available! Your current version is v1.9.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
people = graphlab.SFrame('people_wiki.gl/')

This non-commercial license of GraphLab Create is assigned to vaibhavl@buffalo.edu and will expire on May 18, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-06-13 21:26:51,188 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1465833407.log


In [3]:
people.head()

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


##Adding word_count column

In [4]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])

##Adding tf-idf column

In [7]:
people['tf-idf'] = graphlab.text_analytics.tf_idf(people['word_count'])

In [10]:
elton = people[people['name'] == 'Elton John']
victoria = people[people['name'] == 'Victoria Beckham']
paul = people[people['name'] == 'Paul McCartney']

In [34]:
#Task1-> Compare top words according to word counts to TF-IDF:
elton[['word_count']].stack('word_count',new_column_name=['word','count']).sort('count',ascending=False)

word,count
the,27
in,18
and,15
of,13
a,10
has,9
john,7
he,7
on,6
award,5


In [30]:
##Top_tf-idfs
elton[['tf-idf']].stack('tf-idf',new_column_name=['word','tf-idf']).sort('tf-idf',ascending=False)

word,tf-idf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
tonightcandle,10.9864953892
overallelton,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


In [32]:
##What’s the cosine distance between the articles on ‘Elton John’ and ‘Victoria Beckham’? 
graphlab.distances.cosine(elton['tf-idf'][0],victoria['tf-idf'][0])

0.9567006376655429

In [33]:
##What’s the cosine distance between the articles on ‘Elton John’ and Paul McCartney’? 
graphlab.distances.cosine(elton['tf-idf'][0],paul['tf-idf'][0])

0.8250310029221779

In [38]:
#Task3: Building nearest neighbors models with different input features and setting the distance metric:
# Model using word_count as features
knn_word_count = graphlab.nearest_neighbors.create(people,features=['word_count'],label='name',distance='cosine')

Starting brute force nearest neighbors model training.

In [39]:
# Model using tf-idf as features
knn_tf_idf = graphlab.nearest_neighbors.create(people,features=['tf-idf'],label='name',distance='cosine')

Starting brute force nearest neighbors model training.

In [40]:
knn_word_count.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 2       | 0.00338576  | 65.853ms     |

| Done         |         | 100         | 375.195ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.22044604925e-16,1
0,Cliff Richard,0.16142415259,2
0,Sandro Petrone,0.16822542751,3
0,Rod Stewart,0.168327165587,4
0,Malachi O'Doherty,0.177315545979,5


In [41]:
knn_tf_idf.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 21.114ms     |

| Done         |         | 100         | 443.25ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5


In [42]:
knn_word_count.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 11.823ms     |

| Done         |         | 100         | 327.606ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.22044604925e-16,1
0,Mary Fitzgerald (artist),0.207307036115,2
0,Adrienne Corri,0.214509782788,3
0,Beverly Jane Fry,0.217466468741,4
0,Raman Mundair,0.217695474992,5


In [43]:
knn_tf_idf.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 18.404ms     |

| Done         |         | 100         | 416.678ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5
